---
## 🔧 Step 1: Setup and Imports

In [ ]:
# Import required libraries
import soccerdata as sd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Configure plotting style
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%B %d, %Y')}")

---
## 📊 Step 2: Fetch Current Team Rankings

First, let's see which teams are currently at the top!

In [ ]:
# Initialize ClubElo scraper
print("🔄 Fetching current Elo ratings...")
elo = sd.ClubElo()

# Get current rankings
current_elo = elo.read_by_date()

# Display top 20 teams
print(f"\n🏆 Top 20 Teams by Elo Rating (as of today):\n")
top_20 = current_elo.head(20)
top_20

### 📈 Visualize Current Top 10

In [ ]:
# Create bar chart of current top 10
top_10 = current_elo.head(10)

plt.figure(figsize=(12, 6))
colors = plt.cm.RdYlGn(np.linspace(0.5, 0.9, len(top_10)))
bars = plt.bar(range(len(top_10)), top_10['elo'], color=colors, edgecolor='black', linewidth=1.5)

# Customize
plt.xticks(range(len(top_10)), top_10.index.get_level_values('team'), rotation=45, ha='right')
plt.ylabel('Elo Rating', fontsize=14, fontweight='bold')
plt.title('Current Top 10 Teams by Elo Rating', fontsize=16, fontweight='bold', pad=20)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n💡 The highest-rated team is {top_10.index.get_level_values('team')[0]} with {int(top_10['elo'].iloc[0])} Elo points!")

---
## 📜 Step 3: Historical Evolution of a Single Team

Let's track how one team's performance has evolved over time.

In [ ]:
# Choose a team to analyze (you can change this!)
TEAM_NAME = top_10.index.get_level_values('team')[0]  # Top team
# Or manually set: TEAM_NAME = "Manchester City"  # Change to any team you want

print(f"🔄 Fetching historical data for {TEAM_NAME}...")

# Get team history
team_history = elo.read_team_history(TEAM_NAME)

print(f"✅ Retrieved {len(team_history)} data points")
print(f"📅 Date range: {team_history.index.min().date()} to {team_history.index.max().date()}")
print(f"\n📊 Sample data:")
team_history.head()

### 📈 Plot Single Team Evolution

In [ ]:
# Plot raw data
plt.figure(figsize=(16, 7))

# Raw Elo ratings (more volatile)
plt.plot(team_history.index, team_history['elo'], 
         alpha=0.3, color='lightblue', linewidth=1, label='Raw Elo')

# Smoothed data (30-game rolling average)
smoothed = team_history['elo'].rolling(window=30, center=True).mean()
plt.plot(team_history.index, smoothed, 
         color='darkblue', linewidth=3, label='30-Game Rolling Average')

# Styling
plt.xlabel('Date', fontsize=14, fontweight='bold')
plt.ylabel('Elo Rating', fontsize=14, fontweight='bold')
plt.title(f'{TEAM_NAME} - Elo Rating Evolution', fontsize=18, fontweight='bold', pad=20)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Add horizontal line for current Elo
current_rating = team_history['elo'].iloc[-1]
plt.axhline(y=current_rating, color='red', linestyle='--', alpha=0.5, 
            label=f'Current: {int(current_rating)}')

plt.tight_layout()
plt.show()

# Statistics
print(f"\n📊 Statistics for {TEAM_NAME}:")
print(f"   Current Elo: {int(current_rating)}")
print(f"   Highest Elo: {int(team_history['elo'].max())} on {team_history['elo'].idxmax().date()}")
print(f"   Lowest Elo: {int(team_history['elo'].min())} on {team_history['elo'].idxmin().date()}")
print(f"   Average Elo: {int(team_history['elo'].mean())}")

---
## 🔥 Step 4: Compare Multiple Top Teams

Now let's compare how multiple elite teams have evolved over time!

In [ ]:
# Select teams to compare (top 5 current teams)
NUM_TEAMS = 5
teams_to_compare = current_elo.head(NUM_TEAMS).index.get_level_values('team').tolist()

# Or manually choose teams:
# teams_to_compare = ["Manchester City", "Liverpool", "Real Madrid", "Bayern Munich", "Barcelona"]

print(f"🔄 Fetching data for {NUM_TEAMS} teams...")
print(f"   Teams: {', '.join(teams_to_compare)}\n")

# Fetch data for all teams
team_histories = {}
for team in teams_to_compare:
    try:
        history = elo.read_team_history(team)
        team_histories[team] = history
        print(f"✅ {team}: {len(history)} data points")
    except Exception as e:
        print(f"⚠️  Could not fetch data for {team}: {e}")

print(f"\n✅ Successfully loaded data for {len(team_histories)} teams!")

### 📊 Multi-Team Comparison Chart

In [ ]:
# Create comparison plot
plt.figure(figsize=(18, 9))

# Color palette
colors = plt.cm.Set2(np.linspace(0, 1, len(team_histories)))

# Plot each team with smoothing
smoothing_window = 50  # Adjust for more/less smoothing

for (team, history), color in zip(team_histories.items(), colors):
    # Apply rolling average for smoother lines
    smoothed = history['elo'].rolling(window=smoothing_window, center=True).mean()
    
    plt.plot(smoothed.index, smoothed, 
             linewidth=3, label=team, color=color, alpha=0.8)

# Styling
plt.xlabel('Year', fontsize=14, fontweight='bold')
plt.ylabel('Elo Rating', fontsize=14, fontweight='bold')
plt.title(f'Evolution of Top {len(team_histories)} Teams - Elo Ratings Over Time', 
          fontsize=18, fontweight='bold', pad=20)
plt.legend(loc='best', fontsize=12, framealpha=0.9)
plt.grid(True, alpha=0.3)

# Add shaded regions for context (optional)
plt.axhspan(1900, 2000, alpha=0.05, color='green', label='Strong')
plt.axhspan(1800, 1900, alpha=0.05, color='yellow', label='Good')

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   - Higher lines = stronger teams")
print("   - Rising trends = improving performance")
print("   - Falling trends = declining performance")
print("   - Crossovers = power shifts between teams")

---
## 🎯 Step 5: Recent Performance (Last 5 Years)

Let's zoom in on recent history for a clearer view of current form.

In [ ]:
# Filter to last 5 years
from datetime import datetime, timedelta

five_years_ago = datetime.now() - timedelta(days=5*365)

plt.figure(figsize=(16, 8))

colors = plt.cm.tab10(np.linspace(0, 1, len(team_histories)))

for (team, history), color in zip(team_histories.items(), colors):
    # Filter recent data
    recent = history[history.index >= five_years_ago]
    
    if len(recent) > 0:
        # Light smoothing for recent data
        smoothed = recent['elo'].rolling(window=10, center=True).mean()
        
        plt.plot(smoothed.index, smoothed, 
                linewidth=3.5, label=team, color=color, marker='o', 
                markersize=2, alpha=0.9)

# Styling
plt.xlabel('Date', fontsize=14, fontweight='bold')
plt.ylabel('Elo Rating', fontsize=14, fontweight='bold')
plt.title('Recent Form: Last 5 Years Performance', fontsize=18, fontweight='bold', pad=20)
plt.legend(loc='best', fontsize=11, framealpha=0.95)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show current standings
print("\n🏆 Current Elo Rankings (among selected teams):")
current_ratings = {}
for team, history in team_histories.items():
    current_ratings[team] = history['elo'].iloc[-1]

sorted_teams = sorted(current_ratings.items(), key=lambda x: x[1], reverse=True)
for i, (team, rating) in enumerate(sorted_teams, 1):
    print(f"   {i}. {team}: {int(rating)}")

---
## 📊 Step 6: Trend Analysis - Who's Rising/Falling?

Let's calculate momentum to see which teams are improving or declining.

In [ ]:
# Calculate 1-year change for each team
one_year_ago = datetime.now() - timedelta(days=365)

trends = {}
for team, history in team_histories.items():
    current = history['elo'].iloc[-1]
    
    # Get rating from 1 year ago
    try:
        year_ago_data = history[history.index <= one_year_ago]
        if len(year_ago_data) > 0:
            year_ago = year_ago_data['elo'].iloc[-1]
            change = current - year_ago
            trends[team] = {
                'current': current,
                'year_ago': year_ago,
                'change': change,
                'pct_change': (change / year_ago) * 100
            }
    except:
        pass

# Create DataFrame
trends_df = pd.DataFrame(trends).T
trends_df = trends_df.sort_values('change', ascending=False)

print("📈 1-Year Performance Trends:\n")
print(trends_df.round(1))

# Visualize momentum
plt.figure(figsize=(12, 6))
colors = ['green' if x > 0 else 'red' for x in trends_df['change']]
bars = plt.barh(trends_df.index, trends_df['change'], color=colors, alpha=0.7, edgecolor='black')

plt.xlabel('Elo Point Change (1 Year)', fontsize=13, fontweight='bold')
plt.title('Team Momentum: Rising vs Falling', fontsize=16, fontweight='bold', pad=15)
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(trends_df.iterrows()):
    plt.text(row['change'], i, f" {row['change']:+.0f} ({row['pct_change']:+.1f}%)",
             va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Summary
rising = trends_df[trends_df['change'] > 0]
falling = trends_df[trends_df['change'] < 0]

print(f"\n🔥 Rising teams: {len(rising)}")
print(f"📉 Falling teams: {len(falling)}")

---
## 💾 Step 7: Save Your Visualizations

Let's save the best charts to share!

In [ ]:
# Recreate the main comparison chart for saving
plt.figure(figsize=(18, 10))

colors = plt.cm.Set2(np.linspace(0, 1, len(team_histories)))
smoothing_window = 50

for (team, history), color in zip(team_histories.items(), colors):
    smoothed = history['elo'].rolling(window=smoothing_window, center=True).mean()
    plt.plot(smoothed.index, smoothed, linewidth=3, label=team, color=color, alpha=0.8)

plt.xlabel('Year', fontsize=16, fontweight='bold')
plt.ylabel('Elo Rating', fontsize=16, fontweight='bold')
plt.title(f'Evolution of Top {len(team_histories)} Teams - Elo Ratings', 
          fontsize=20, fontweight='bold', pad=20)
plt.legend(loc='best', fontsize=14, framealpha=0.95)
plt.grid(True, alpha=0.3)

# Save
output_file = '../output/team_evolution_comparison.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✅ Chart saved to: {output_file}")

plt.show()

# Save data to CSV
if len(team_histories) > 0:
    # Combine all team data
    combined_data = pd.DataFrame()
    for team, history in team_histories.items():
        temp = history[['elo']].copy()
        temp.columns = [team]
        combined_data = pd.concat([combined_data, temp], axis=1)
    
    csv_file = '../output/team_evolution_data.csv'
    combined_data.to_csv(csv_file)
    print(f"✅ Data saved to: {csv_file}")

print("\n🎉 All outputs saved successfully!")

---
## 🎨 Step 8: Customize - Try Your Own Teams!

Now it's your turn! Modify the teams and create your own analysis.

In [ ]:
# YOUR CUSTOM ANALYSIS HERE!
# Try these examples or create your own:

# Example 1: Compare historical rivals
# my_teams = ["Barcelona", "Real Madrid"]

# Example 2: Premier League big 6
# my_teams = ["Manchester City", "Liverpool", "Chelsea", "Arsenal", "Manchester United", "Tottenham"]

# Example 3: National teams (if available)
# my_teams = ["Argentina", "Brazil", "Germany", "France"]

# Uncomment and run:
# my_teams = ["YOUR", "TEAMS", "HERE"]
# 
# plt.figure(figsize=(16, 8))
# for team in my_teams:
#     try:
#         history = elo.read_team_history(team)
#         smoothed = history['elo'].rolling(window=50, center=True).mean()
#         plt.plot(smoothed.index, smoothed, linewidth=3, label=team, alpha=0.8)
#     except:
#         print(f"Could not load {team}")
# 
# plt.legend()
# plt.title('My Custom Team Comparison')
# plt.xlabel('Year')
# plt.ylabel('Elo Rating')
# plt.grid(True, alpha=0.3)
# plt.show()

print("✨ Ready for your custom analysis!")

---
## 🎓 What You've Learned!

Congratulations! You've successfully:

✅ Fetched historical Elo rating data  
✅ Created time-series visualizations  
✅ Compared multiple teams over time  
✅ Applied data smoothing techniques  
✅ Analyzed trends and momentum  
✅ Saved professional-quality charts  

## 🚀 Next Steps:

1. **Experiment**: Try different teams and time periods
2. **Combine**: Add match results or tournament data
3. **Share**: Show your visualizations to friends!
4. **Learn More**: Check out the other project notebooks

## 💡 Ideas for Enhancement:

- Add markers for major tournaments (World Cup, Champions League)
- Create an animated timeline showing evolution
- Compare leagues instead of individual teams
- Predict future Elo ratings using trends
- Add interactive Plotly charts for web sharing

---

**Great work! You're now a soccer data visualization pro! ⚽📊🏆**